# <center> "Online" Clustering with HDBSCAN </center>

The purpose of this notebook is to look into speedups for the HDBSCAN algorithm when we already have a clustering and we want to add new data points to it. This is the situation that we will have with the reduced embeddings, where we will cluster, and then new embeddings will be added to the original dataset, and we want to see how the clusters change and evolve over time.

## Step 1: Core Distances

The first issue we come across is that when we add new data points to the dataset, we need to calculate the core distances not only for the new data points, but also for the old data points. This is because the core distances are calculated based on the k-nearest neighbors, and the new data points may change the k-nearest neighbors of some of the old data points. For reference, the core distance is defined as
$$\text{core}_k(v) = d(v,q)$$
where $q$ is the $k$-th nearest neighbor of $v$.

The idea here is that we can speed up computation by just looking at updating core distances for the old data points that we know will change (i.e. distance between points is less than old core distance) instead of recomputing the core distances for all the old data points. This way, we can avoid recomputing the core distances for the old data points that are not affected by the new data points.

In the worst case scenario, we would have to recompute the core distances for all the old data points, but since the alternative is recomputing all the old core distances anyway, this should help speed up computation somewhat. The only extra computation we need to perform over the original algorithm is $np$ comparisons between distances and old core distances, where $n$ is the number of new data points and $p$ is the number of old data points.

I also think that the HDBSCAN algorithm would benefit from a different tree implementation for the nearest neighbors search over the current KD-tree implementation. The R*-tree or PH-tree structures may better allow for KNN search and new inserts over the KD-tree implementation.

## Step 2: Minimum Spanning Tree

The next issue that we come across is that when we change core distances, we also potentially change the reachability distances and thus the minimum spanning tree. Note that the reachability distance is defined as
$$d_r(v,w) = \text{max}\{\text{core}_k(v), \text{core}_k(w), d(v,w)\}$$

My theory is that we can use parts of the previously computed minimum spanning tree as a starting forest for the new minimum spanning tree. This is based off of the idea that when adding new vertices to an existing MST, we can run Prim's (or another MST algorithm) starting from the existing MST and adding the new vertices and edges ([here](https://stackoverflow.com/questions/52168792/update-minimum-spanning-tree-after-a-new-vertex-is-added)). This should be faster than recomputing the entire MST from scratch. Another response also mentions algorithms that exist for updating MSTs, such as those in [this paper](https://www.sciencedirect.com/science/article/pii/0022000078900223).

In [this response](https://stackoverflow.com/a/9934785) to a different question, we may be able to handle the case of old points that have had their core distances updated, especially when we consider that the addition of a point can only decrease the core distance between two points and thus the reachability distance. Thus, we have only two cases to consider of the four outlined in the response.

The problem is that since the core distances for some of the old points

Let $G = (V, E)$ be the graph with vertices $V$ and edges $E$ where $V$ is the set of data points and $E$ is the set of edges between the data points. Let $T = (V, E_T)$ be the minimum spanning tree of $G$ created by the HDBSCAN algorithm for some $k$ where $E_T$ is the set of edges in the minimum spanning tree. Note that by the HDBSCAN algorithm, T was created with weights defined by the reachability distance metric. Let $d_r(a,b)$ be the reachability distance and $d(a,b)$ be the eucliden distance between two points $a$ and $b$.

Let $z$ be a new data point that is added to the dataset. Define $S = \{v \in V | d(v,z) < \text{core}_k(v)\}$, where $\text{core}_k(v)$ is the core distance of a point $v \in V$ and $V$ is the original set of data points. We have the following:

1. $\forall v \notin S$, $\text{core}_k(v) = \text{core}_{k,\text{old}}(v)$. *Explanation*: $\forall v \notin S$, $d(v,z) \geq \text{core}_k(v)$; therefore, $z$ is either not in the $k$-nearest neighbors of $v$ or is tied with another point as the $k\text{th}$-nearest neighbor.
2. $\forall v \in V$, $\text{core}_k(v) \leq \text{core}_{k,\text{old}}(v)$. *Explanation*: $\forall v \in S$, $d(v,z) < \text{core}_k(v)$; therefore, $z$ is in the $k$-nearest neighbors of $v$ and is closer than the $k\text{th}$-nearest neighbor. The core distance may remain the same if $v$ has multiple points tied as the $k\text{th}$-nearest neighbor, or if $v \notin S$.
3. $\forall v,w \notin S$, $d_r(v,w) = d_{r,\text{old}}(v,w)$. *Explanation*: If $v,w \notin S$, then the core distances of $v$ and $w$ have not changed, and thus the reachability distance between $v$ and $w$ has not changed.
4. $\forall v,w \in V$, $d_r(v,w) \leq d_{r,\text{old}}(v,w)$. *Explanation*: The reachability distance is calculated as the maximum of the core distance and the distance between the two points. Since the core distance cannot increase, the reachability distance cannot increase either.

Define the new MST with the new data point $z$ as $T' = (V \cup \{z\}, E_{T'})$ where $E_{T'} \subset E$ is the set of edges in the new minimum spanning tree. 

I theorize that:

* For $(v,w) \notin E_T$, if $d_r(v,w) = d_{r,\text{old}}(v,w)$, then $e \notin E_{T'}$. (TRUE, proven)
* From of the above and (3), for $(v,w) \notin E_T$, if $v,w \notin S$, then $e \notin E_{T'}$

I initially thought that for $(v,w) \in E_T$, if $d_r(v,w) = \text{max}(\text{core}_k(v),\text{core}_k(w))$, then $e \in E_{T'}$; however, there are counterexamples. For example, consider a cycle of $G$ with a largest weight edge $e$ which is omitted from $E_T$. Suppose that a point $z$ is added which is not in the cycle, but doing so reduces the core distance of a vertex connected to $e$ such that the weight of $e$ is now less than other edges in the cycle. Then $e$ will be included in $E_{T'}$, forcing another edge to be omitted from $E_{T'}$ in order to avoid creating a cycle.

The plan was to use the above to initialize Boruvka's algorithm with the subtrees generated by the edges that were maintained from the original minimum spanning tree. However, since the expression is false, we will just initialize Boruvka's algorithm normally. However, from our true case (For $(v,w) \notin E_T$, if $d_r(v,w) = d_{r,\text{old}}(v,w)$, then $e \notin E_{T'}$), we can exclude edges in $G$ that were not in the original MST and were not affected by the new data point.

Alternatively, we can try to implement a graph update algorithm from the aforementioned paper; however, these are $O(np)$ where $n$ is the number of original data points and $p$ is the number of updated core distances plus new edges. It may also need to be done sequentially, whereas the Boruvka's algorithm can be done in parallel.

I think Boruvka's may be the way to go here. We need to look into how Boruvka's algorithm initialized in the HDBSCAN implementation, since I can't imagine they provide every edge in the graph. It may also be a good idea to create a strategy for breaking ties when creating the MST since the reachability distance ends up creating a lot of ties.